# Imports

In [1]:
import pandas as pd

# Defining a handy function

In [11]:
def TransformColumns(df):
    """Replaces the spaces with underlines and converts to lowecase the column names of a given DataFrame `df`.
    
    Parameters
    ----------
    df : DataFrame with messy column names
    
    Returns
    -------
    None
    
    Raises
    ------
    AssertionError
        * If `df` is not a ``pandas.DataFrame``.
    """
    
    try:
        assert type(df) is pd.core.frame.DataFrame, "Invalid input. Must be a DataFrame."
        df.columns = [c.lower().replace(' ', '_') for c in df.columns]
        return None
    except AssertionError as msg:
        print(msg)
        return None

# Implementing solution

### Declaring variables

In [ ]:
dictMonths = {1: 'jan', 2:'fev', 3:'mar', 4:'abr', 5:'mai', 6:'jun', 7:'jul', 8:'ago', 9:'set', 10:'out', 11:'nov', 12:'dez'}

varMonth = input("Enter the first MONTH of the forecast that should be reviewed: ")
varYear = input("Enter the YEAR of the first month that should be reviewed: ")

varQuery = 'ano == '+str(varYear)+' or (ano == '+str(int(varYear) - 1)+' and mes >= '+str(varMonth)+')'

### Importing and preparing DataFrames

Here all DataFrames used in this solution are imported and transformed with the function TransformColumns, previously declared.

In [ ]:
dfGroupAnalyst = pd.read_excel('./Inputs/0_Group per analyst.xls')
dfProd = pd.read_excel('./Inputs/1_Products.xls', sheet_name='Plan1', usecols='A:D,F,G,K')
dfStockStores = pd.read_excel('./Inputs/1_ESTOQUE REDE.xls', sheet_name='Plan1')
dfStockECMandDC = pd.read_excel('./Inputs/1_STOCK DC AND ECM.xls', sheet_name='Plan1')
dfVendas = pd.read_excel('./Inputs/2_VENDAS.xlsx', usecols='A, D, L, N, O')
dfDFUAntiga = pd.read_excel('./Inputs/2_DFU ANTIGA.xlsx')

TransformColumns(dfProd)
TransformColumns(dfStockStores)
TransformColumns(dfECMandDC)
TransformColumns(dfVendas)

From our Product Dataframe (dfProd), let's filter out all products that have been discontinued and that are unavailabe with no reorder, as well as all products where 'griffe' is different than Brand A. Due to business specifics, let's also filter out Groups E and AT.

In [ ]:
dfProd = dfProd.loc[dfProd.linha != 'DISCONTINUED', :]
dfProd = dfProd.loc[dfProd.linha != 'UNAVAILABLE - NO REORDER', :]
dfProd = dfProd.loc[dfProd.griffe == 'Brand A', :]
dfProd = dfProd.loc[dfProd.grupo_produto != 'Group E', :]
dfProd = dfProd.loc[dfProd.grupo_produto != 'Group AT', :]


In [ ]:
With the stock 

In [ ]:
dfStockECMandDC['estoque_cd'] = dfStockECMandDC['disponivel_venda'] + dfStockECMandDC['movimentação_cd']

dfECM = dfStockECMandDC.loc[dfStockECMandDC.filial == 'ECOMMERCE', :]
dfDC = dfStockECMandDC.loc[dfStockECMandDC.filial == 'DC', :]




dfProd = dfProd.merge(dfGroupAnalyst, on='grupo_produto', how='left')








In [ ]:
### Handl

In [ ]:

dfVendasLTM = dfVendas.query(varQuery)
pivotVendasLTM = dfVendasLTM.copy(deep=True)
pivotVendasLTM['ano_e_mes'] = pivotVendasLTM.apply(lambda x: str(dictMeses[x['mes']])+'/'+str(x['ano']), axis=1)
pivotVendasLTM = pivotVendasLTM.pivot_table(values='venda', index='produto', columns='ano_e_mes', aggfunc='sum')

mesmin = dfVendasLTM['data_registro'].min().month
anomin = dfVendasLTM['data_registro'].min().year

dfDFU = dfProd.copy(deep=True)


for m in range(0, 12):
    if mesmin + m <= 12:
        mes = mesmin + m
        ano = anomin
    else:
        mes = mesmin + m -12
        ano = anomin +1

    mes_ano = str(dictMeses[mes])+'/'+str(ano)
    dfDFU = dfDFU.merge(pivotVendasLTM[[mes_ano]], left_on='produto', right_index=True, how='left')



dfDFU = dfDFU.merge(dfStockStores[['produto', 'estoque']], on='produto', how='left')
dfDFU.rename(index=str, columns={"estoque": "rede"}, inplace=True)

dfDFU = dfDFU.merge(dfDC[['produto', 'estoque_cd']], on='produto', how='left')
dfDFU.rename(index=str, columns={"estoque_cd": "cd"}, inplace=True)

dfDFU = dfDFU.merge(dfECM[['produto', 'estoque_cd']], on='produto', how='left')
dfDFU.rename(index=str, columns={"estoque_cd": "ecm"}, inplace=True)

dfDFU = dfDFU.fillna(0)
dfDFU['estoque_rede+cd'] = dfDFU['rede'] + dfDFU['cd']





mesesFuturos =[]

for m in range(0, 14):
    if mesmin + m <= 12:
        mes = mesmin + m
        ano = anomin + 1
    else:
        mes = mesmin + m -12
        ano = anomin + 2
    mes_ano = str(dictMeses[mes])+'/'+str(ano)
    mesesFuturos.append(mes_ano)


for m in mesesFuturos[:-1]:
    dfDFU[m] = dfDFU.apply(lambda x: dfDFUAntiga.loc[(dfDFUAntiga['produto'] == x['produto'])][m].sum(), axis=1)
for m in mesesFuturos[-1:]:
    dfDFU[m] = dfDFU[dfDFU.columns.tolist()[-1]]




dfDFU.to_excel("DFUNova.xlsx", sheet_name='Base', index=False)